# DataPath Example 2
This notebook gives a very basic example of how to access data. 
It assumes that you understand the concepts presented in the 
example 1 notebook.

In [ ]:
# Import deriva modules
from deriva.core import ErmrestCatalog, get_credential

In [ ]:
# Connect with the deriva catalog
protocol = 'https'
hostname = 'www.facebase.org'
catalog_number = 1
credential = get_credential(hostname)
catalog = ErmrestCatalog(protocol, hostname, catalog_number, credential)

In [ ]:
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()

## DataPaths
The `PathBuilder` object allows you to begin `DataPath`s from the base `Table`s. A `DataPath` begins with a `Table` (or an `TableAlias` to be discussed later) as its "root" from which one can "`link`", "`filter`", and fetch its "`entities`".

### Start a path rooted at a table from the catalog
We will reference a table from the PathBuilder `pb` variable from above. Using the PathBuilder, we will reference the "isa" schema, then the "dataset" table, and from that table start a path.

In [ ]:
path = pb.schemas['isa'].tables['dataset'].path

We could have used the more compact dot-notation to start the same path.

In [ ]:
path = pb.isa.dataset.path

### Getting the URI of the current path
All DataPaths have URIs for the referenced resources in ERMrest. The URI identifies the resources which are available through "RESTful" Web protocols supported by ERMrest.

In [ ]:
print(path.uri)

## ResultSets
The data from a DataPath are accessed through a pythonic container object, the `ResultSet`. The `ResultSet` is returned by the DataPath's `entities()` and other methods.

In [ ]:
results = path.entities()

### Fetch entities from the catalog
Now we can get entities from the server using the ResultSet's `fetch()` method.

In [ ]:
results.fetch()

`ResultSet`s behave like python containers. For example, we can check the count of rows in this ResultSet.

In [ ]:
len(results)

**Note**: If we had not explicitly called the `fetch()` method, then it would have been called implicitly on the first container operation such as `len(...)`, `list(...)`, `iter(...)` or get item `[...]`.

### Get an entity
To get one entity from the set, use the usual container operator to get an item.

In [ ]:
results[9]

### Get a specific attribute value from an entity
To get one attribute value from an entity get the item using its `Column`'s `name` property.

In [ ]:
dataset = pb.schemas['isa'].tables['dataset']
print (results[9][dataset.accession.name])

## Fetch a Limited Number of Results
To set a limit on the number of results to be fetched from the catalog, use the explicit `fetch(limit=...)` method with the desired upper limit to fetch from the catalog.

In [ ]:
results.fetch(limit=3)
len(results)

### Iterate over the ResultSet
`ResultSet`s are iterable like a typical container.

In [ ]:
for entity in results:
    print(entity[dataset.accession.name])

## Convert to Pandas DataFrame
ResultSets can be transformed into the popular Pandas DataFrame.

In [ ]:
results.dataframe

## Selecting Attributes
It is also possible to fetch only a subset of attributes from the catalog. The `attributes(...)` method accepts a variable argument list followed by keyword arguments. Each argument must be a `Column` object from the table's `columns` container.

### Renaming selected attributes
To rename the selected attributes, use "named" (a.k.a., "keyword") arguments in the method. For example, `attributes(..., new_name=table.column)` will rename `table.column` with `new_name` in the entities returned from the server. (It will not change anything in the stored catalog data.) Note that in python, the named arguments _must come after_ positional arguments.

In [ ]:
results = path.attributes(dataset.accession, dataset.title, is_released=dataset.released).fetch(limit=5)

### Convert to list
Now we can look at the results from the above fetch. To demonstrate a different access mode, we can convert the entities to a standard python list and dump to the console.